In [12]:
import numpy as np
import pandas as pd
import re

[Requête SQL data.stackexchange.com ici](https://data.stackexchange.com/stackoverflow/query/1824062/top-50k-relevant-questions)

⇒ compromis fait entre qualité maximale et contraintes (≥ 5 tags)
- pour qualité, score ≥ 10 et > 3 réponses
- dates jusqu’à 2009, cohérent
- reste à spectre large pour tout type de question

seules features indispensables & RGPD dans la requête

In [13]:
df_raw = pd.read_csv("data/2024-03-11 QueryResults.csv")

# Nettoyage des colonnes

In [14]:
# remove useless features
df_raw.drop(columns=["Score", "AnswerCount", "CreationDate"], inplace=True)
# change Tags string to list
df_raw["Tags"] = df_raw["Tags"].apply(lambda x: x[1:-2].split("><"))

df_raw

,Title,Body,Tags
0,std::shared_mutex::unlock_shared() blocks even...,<p>My team has encountered a deadlock that I s...,"[c++, windows, multithreading, stl, shared-loc]"
1,"What is the correct output of sizeof(""string"")?","<p>On a microcontroller, in order to avoid loa...","[c, language-lawyer, sizeof, string-literals, ..."
2,Problem loading external scripts like jQuery,<p>I'm facing a problem since this morning wit...,"[javascript, html, jquery, google-apps-script,..."
3,Does a const char* literal string persistently...,<p>I have functions like the following:</p>\n<...,"[c++, constants, constexpr, lifetime, null-ter..."
4,WillPopScope is deprecated in Flutter,<p>'WillPopScope' is deprecated and shouldn't ...,"[flutter, dart, mobile, cross-platform, deprec..."
...,...,...,...
49995,The activator for bundle is invalid,<p>I'm trying to create a simple plugin in ecl...,"[java, eclipse, eclipse-plugin, osgi, osgi-bundl]"
49996,Pre-allocate space for C++ STL queue,<p>I'm writing a radix sort algorithm using qu...,"[c++, performance, memory, stl, queu]"
49997,Adjusting the positions of the labels in a UIT...,<p>I'm using a UITableViewCell with UITableVie...,"[iphone, objective-c, cocoa-touch, iphone-sdk-..."
49998,Padding on StackPanel?,<p>I am trying to set padding of a StackPanel ...,"[wpf, xaml, layout, padding, stackpane]"


🚧 [HTML Parser](https://stackoverflow.com/questions/59660495/python-html-extract-text-into-list) : (cf. [question SoF](https://stackoverflow.com/questions/59660495/python-html-extract-text-into-list))  
[RegEx JL](https://www.notion.so/julmat/RegEx-expressions-r-guli-res-70fbbcb177ee476ba9a5ae011d14fe6f) & [RegEx 101](https://regex101.com/?flavor=python&flags=gm)

In [15]:
print(df_raw["Body"][0])

<p>My team has encountered a deadlock that I suspect is a bug in the Windows implementation of SRW locks. The code below is a distilled version of real code. Here's the summary:</p>
<ol>
<li>Main thread acquires exclusive lock</li>
<li>Main thread creates N children threads</li>
<li>Each child thread
<ol>
<li>Acquires a shared lock</li>
<li>Spins until all children have acquired a shared lock</li>
<li>Releases the shared lock</li>
</ol>
</li>
<li>Main thread releases exclusive lock</li>
</ol>
<p>Yes this could be done with std::latch in C++20. That's not the point.</p>
<p>This code works <em>most</em> of the time. However roughly 1 in 5000 loops it deadlocks. When it deadlocks exactly 1 child successfully acquires a shared lock and N-1 children are stuck in <code>lock_shared()</code>. On Windows this function calls into <code>RtlAcquireSRWLockShared</code> and blocks in <code>NtWaitForAlertByThreadId</code>.</p>
<p>The behavior is observed when used <code>std::shared_mutex</code> direc

In [16]:
# remove code tags from "Body" column
df_raw["Body"] = df_raw["Body"].apply(lambda x: re.sub(r"<code>.*?<\/code>", "", x, flags=re.S))
# remove img tags from "Body" column
df_raw["Body"] = df_raw["Body"].apply(lambda x: re.sub(r"<img.*?>", "", x))
# remove html tags from "Body" column
df_raw["Body"] = df_raw["Body"].apply(lambda x: re.sub(r"<.*?>", "", x))
# remove newlines from "Body" column
df_raw["Body"] = df_raw["Body"].apply(lambda x: re.sub(r"\n", " ", x))
# lowercase "Body" column
df_raw["Body"] = df_raw["Body"].apply(lambda x: x.lower())
# keep only letters and spaces in "Body" column
df_raw["Body"] = df_raw["Body"].apply(lambda x: re.sub(r"[^a-z ]", "", x))
# remove multiple spaces from "Body" column
df_raw["Body"] = df_raw["Body"].apply(lambda x: re.sub(r" +", " ", x))

In [17]:
print(df_raw["Body"][0])

my team has encountered a deadlock that i suspect is a bug in the windows implementation of srw locks the code below is a distilled version of real code heres the summary main thread acquires exclusive lock main thread creates n children threads each child thread acquires a shared lock spins until all children have acquired a shared lock releases the shared lock main thread releases exclusive lock yes this could be done with stdlatch in c thats not the point this code works most of the time however roughly in loops it deadlocks when it deadlocks exactly child successfully acquires a shared lock and n children are stuck in on windows this function calls into and blocks in the behavior is observed when used directly or simply calling functions directly a raymond chen post asks about this exact behavior but user error is blamed this looks like an srw bug to me its maybe worth noting that if a child doesnt attempt to latch and calls that this will wake its blocked siblings there is nothing

In [18]:
df_raw

,Title,Body,Tags
0,std::shared_mutex::unlock_shared() blocks even...,my team has encountered a deadlock that i susp...,"[c++, windows, multithreading, stl, shared-loc]"
1,"What is the correct output of sizeof(""string"")?",on a microcontroller in order to avoid loading...,"[c, language-lawyer, sizeof, string-literals, ..."
2,Problem loading external scripts like jQuery,im facing a problem since this morning with we...,"[javascript, html, jquery, google-apps-script,..."
3,Does a const char* literal string persistently...,i have functions like the following and im pla...,"[c++, constants, constexpr, lifetime, null-ter..."
4,WillPopScope is deprecated in Flutter,willpopscope is deprecated and shouldnt be use...,"[flutter, dart, mobile, cross-platform, deprec..."
...,...,...,...
49995,The activator for bundle is invalid,im trying to create a simple plugin in eclipse...,"[java, eclipse, eclipse-plugin, osgi, osgi-bundl]"
49996,Pre-allocate space for C++ STL queue,im writing a radix sort algorithm using queues...,"[c++, performance, memory, stl, queu]"
49997,Adjusting the positions of the labels in a UIT...,im using a uitableviewcell with uitableviewcel...,"[iphone, objective-c, cocoa-touch, iphone-sdk-..."
49998,Padding on StackPanel?,i am trying to set padding of a stackpanel but...,"[wpf, xaml, layout, padding, stackpane]"
